# Consolidação de Arquivos - Múltipos Layouts e Extensões (OBT)

### Criação do modelo com todos os atributos possíveis

In [1]:
import duckdb
import pandas as pd

In [21]:
df_base_atributos = pd.read_csv(
    "../resultados/detalhes/consolidacao_detalhes.csv",
    sep=";",
    dtype="str"
)

In [22]:
df_base_atributos.head(5)

,caminho,arquivo,coluna,extensao,amostra,quantidade,aba,observacoes,status
0,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,DEPARTAMENTO,.xlsx,RH;TI,3,Sheet1,NaN,OK
1,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,ID,.xlsx,1;2,3,Sheet1,NaN,OK
2,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,IDADE,.xlsx,30;25,3,Sheet1,NaN,OK
3,../arquivos/dir1/informacao-departamento.xlsx,informacao-departamento.xlsx,NOME,.xlsx,Ana;Bruno,3,Sheet1,NaN,OK
4,../arquivos/dir1/informacao-erro.png,informacao-erro.png,NaN,.png,NaN,NaN,NaN,Erro: Formato de arquivo não suportado,ERRO


In [31]:
query = """
SELECT
    coluna,
    amostra
FROM (
    SELECT 
            ROW_NUMBER() OVER (PARTITION BY coluna ORDER BY coluna, amostra) AS ID,
            coluna,
            split_part(amostra, ';', 1) amostra
    FROM df_base_atributos
    WHERE coluna IS NOT NULL
    ORDER BY coluna
    ) T1
WHERE ID = 1;
"""

In [61]:
with duckdb.connect(database=":memory:", read_only=False) as conexao:
    df_modelo_atributos = conexao.sql(query).df()

In [53]:
df_modelo_atributos

,coluna,amostra
0,CARGO,Gerente
1,CPF,111.111.111-11
2,DEPARTAMENTO,RH
3,EMAIL,ana@example.com
4,EMPRESA,Empresa A
5,ENDEREÇO,SEM AMOSTRA
6,ESTADO,SP
7,ID,1
8,IDADE,30
9,NOME,Ana


In [62]:
df_modelo_atributos = df_modelo_atributos.pivot(columns="coluna", values="amostra")

In [63]:
df_modelo_atributos.ffill().bfill()
#df_modelo_atributos.ffill().bfill().iloc[0]

coluna,CARGO,CPF,DEPARTAMENTO,EMAIL,EMPRESA,ENDEREÇO,ESTADO,ID,IDADE,NOME,PROJETO,SALARIO,STATUS,TELEFONE
0,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
1,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
2,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
3,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
4,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
5,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
6,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
7,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
8,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789
9,Gerente,111.111.111-11,RH,ana@example.com,Empresa A,SEM AMOSTRA,SP,1,30,Ana,Projeto SWD,5000,Ativo,123456789


In [64]:
df_modelo_atributos = df_modelo_atributos.ffill().bfill().iloc[0]

In [65]:
(
    df_modelo_atributos.to_json(
        "../modelo/base/layout_modelo.json",
        orient="records",
        force_ascii=False
    )
)

# Referências

+ https://docs.pydantic.dev/latest/integrations/datamodel_code_generator/